In [1]:
import tensorflow as tf

from tensorflow import keras


print(tf.__version__)

2.0.0


In [2]:
class Xception_entry_flow(keras.layers.Layer):
    def __init__(self,filters_1,activation,size_1,**kwargs):
        super().__init__(**kwargs)
            
        self.main_layer = [
            keras.layers.SeparableConv2D(filters = filters_1,kernel_size = size_1,activation = 'relu',padding='same',use_bias=False),
            keras.layers.SeparableConv2D(filters = filters_1,kernel_size = size_1,padding='same',use_bias=False),
            keras.layers.MaxPool2D(pool_size= 3,strides=2,padding='same')

        ]

        self.skip_layer = [

            keras.layers.Conv2D(filters = filters_1,kernel_size=1,strides=2,padding='same',use_bias = False)

        ]

        self.activation = keras.activations.get(activation)
            
            
    def call(self,inputs):
        Z_1 = inputs
        for layer in self.main_layer:
            Z_1 = layer(Z_1)
            
        
        Z_2 = inputs
        for layer in self.skip_layer:
            Z_2 = layer(Z_2)
        


        return self.activation(Z_1+Z_2)
    
class Xception_middle_flow(keras.layers.Layer):
    def __init__(self,filters_1,activation,size_1,**kwargs):
        super().__init__(**kwargs)
            
        self.main_layer = [
            keras.layers.SeparableConv2D(filters = filters_1,kernel_size = size_1,activation = 'relu',padding='same',use_bias=False),
            keras.layers.SeparableConv2D(filters = filters_1,kernel_size = size_1,activation='relu',padding='same',use_bias=False),
            keras.layers.SeparableConv2D(filters = filters_1,kernel_size = size_1,padding='same',use_bias=False)


        ]

        self.skip_layer = [


        ]

        self.activation = keras.activations.get(activation)
            
            
    def call(self,inputs):
        Z_1 = inputs
        for layer in self.main_layer:
            Z_1 = layer(Z_1)
            
        
        Z_2 = inputs
        for layer in self.skip_layer:
            Z_2 = layer(Z_2)
        


        return self.activation(Z_1+Z_2)

    
class Xception_exit_flow(keras.layers.Layer):
    def __init__(self,filters_1,filters_2,activation,size_1,**kwargs):
        super().__init__(**kwargs)
            
        self.main_layer = [
            keras.layers.SeparableConv2D(filters = filters_1,kernel_size = size_1,activation = 'relu',padding='same',use_bias=False),
            keras.layers.SeparableConv2D(filters = filters_2,kernel_size = size_1,padding='same',use_bias=False),
            keras.layers.MaxPool2D(pool_size= 3,strides=2,padding='same')

        ]

        self.skip_layer = [

            keras.layers.Conv2D(filters = filters_2,kernel_size=1,strides=2,padding='same',use_bias = False)

        ]

        self.activation = keras.activations.get(activation)
            
            
    def call(self,inputs):
        Z_1 = inputs
        for layer in self.main_layer:
            Z_1 = layer(Z_1)
            
        
        Z_2 = inputs
        for layer in self.skip_layer:
            Z_2 = layer(Z_2)
        


        return self.activation(Z_1+Z_2)   
    






In [3]:

model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32,kernel_size=3,strides = 2,activation='relu',input_shape =[299,299,3]))
model.add(keras.layers.Conv2D(64,kernel_size=3,strides=1,activation='relu'))
### entry flow section

for filter_value in [128,256,728]:
    
    model.add(Xception_entry_flow(filters_1=filter_value,activation='relu',size_1=3))
for i in range(8):
    
    model.add(Xception_middle_flow(filters_1=728,activation='relu',size_1=3))


    
model.add(Xception_exit_flow(filters_1=728,filters_2=1024,activation=None,size_1=3))

model.add(keras.layers.SeparableConv2D(filters = 1536,kernel_size = 3,activation = 'relu',padding='same',use_bias=False))
model.add(keras.layers.SeparableConv2D(filters = 2048,kernel_size = 3,activation = 'relu',padding='same',use_bias=False))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(1000,activation='softmax'))



2022-07-09 17:24:44.956107: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-09 17:24:44.956671: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [4]:
model.build()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 149, 149, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
xception_entry_flow (Xceptio (None, 74, 74, 128)       34496     
_________________________________________________________________
xception_entry_flow_1 (Xcept (None, 37, 37, 256)       134528    
_________________________________________________________________
xception_entry_flow_2 (Xcept (None, 19, 19, 728)       911576    
_________________________________________________________________
xception_middle_flow (Xcepti (None, 19, 19, 728)       1609608   
_________________________________________________________________
xception_middle_flow_1 (Xcep (None, 19, 19, 728)       1